# Neuronale Netzwerke in Python

Dieses Jupyter Notebook zeigt, wie man einen Datensatz für das Trainieren eines künstlichen neuronalen Netzes vorbereitet und wie man ein einfaches Feed-Forward-Netz in Python programmiert.

## Der Titanic-Datensatz
Zuerst werden alle Module importiert, die im Programm benutzt werden.

In [ ]:
# Importiere die nötigen Module
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import keras

In diesem Jupyter Notebook arbeiten wir mit dem _Titanic_ Datensatz, der Informationen inklusive Überlebensstatus über 891 Passagiere der Titanic enthält. Dieser Datensatz stamm aus einer <a href="https://www.kaggle.com/competitions/titanic/overview"> _Kaggle Challenge_</a> zur Vorhersage des Überlebens der Passagiere. Also besteht unsere Klassifikationsaufgabe darin, anhand der Informationen über die Passagiere ihre Überlebenschance zu bestimmen. Übrigens, die Hauptquelle, aus der die meisten Informationen über die Passagiere der Titanic entnommen wurden, ist die <a href="https://www.encyclopedia-titanica.org/"> _Encyclopedia Titanica_</a>.

Wir laden aber zuerst den _Titanic_-Datensatz und speichern ihn in der Variable _titanic_. Dieser Datensatz ist im Modul <a href="https://seaborn.pydata.org/"> _seaborn_</a> enthalten. Der Datensatz wird wie alle Datensätze aus diesem Modul in der Datenstruktur <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html"> _DataFrame_</a> gespeichert.

In [ ]:
# Lade den Titanic-Datensatz
titanic = sns.load_dataset("titanic")

Mit der Methode _info()_ kann eine kurze Zusammenfassung des DateFrames einschließlich der Merkmalsbezeichnungen ausgegeben werden.

In [ ]:
# Gebe eine kurze Zusammenfassung des DataFrames aus
print(titanic.info())

Da einige Merkmale ähnliche Informationen über die Passagiere zu enthalten scheinen, lassen wir uns am besten mit der Funktion _head()_ die ersten fünf Zeilen des Datensatzes anzeigen.

In [ ]:
# Gebe die ersten fünf Zeilen des Datensatzes aus
print(titanic.head())

In der Ausgabe sieht man, dass die Merkmale _survived_ und _alive_ die gleiche Information über den Überlebensstatus der Passagiere enthalten. Im Merkmal _embarked_ sind die Einschiffungshäfen (_embark_town_) in der abgekürzten Form gespeichert. In den beiden Merkmalen _pclass_ und _class_ ist die Passagierklasse einmal als numerischer und einmal als kategorischer Attributwert erfasst und soll über den sozioökonomischen Status der Passagiere Aufschluss geben.

Die Bedeutung der Merkmale _sibsp_ und _parch_ kann auf der Internetseite der <a href="https://www.kaggle.com/competitions/titanic/overview"> _Kaggle Challenge_</a> nachgeschaut werden. Dort erfahren wir, dass im Merkmal _sibsp_ die Anzahl der Geschwister bzw. der Ehepartner an Bord erfasst ist. Im Merkmal _parch_ ist die Anzahl der Eltern bzw. die Anzahl der Kinder an Bord der Titanic des jeweiligen Passagiers gespeichert.

Auf der Internetseite der <a href="https://www.kaggle.com/competitions/titanic/overview"> _Kaggle Challenge_</a> steht leider nichts über die Bedeutung der Merkmale _who_ und _adult_male_. Während wir die Bedeutung des Merkmals _adult_male_ aus seiner Bezeichnung ableiten können, können uns nur die Einträge des Merkmals _who_ etwas über seine Bedeutung verraten. Mit der Funktion _unique()_ können wir eindeutige Werte des Merkmals who ausgeben lassen.

In [ ]:
# Gebe eindeutige Werte des Merkmals 'who' aus
print(titanic['who'].unique())

Laut Ausgabe werden mit Hilfe dieses Merkmals die Passagiere in Kategorien _Mann_, _Frau_ und _Kind_ unterteilt. Da wir das Merkmal _Alter_ wegen fehlender Werte nicht benutzen können, können die Einträge des Merkmals who uns zumindest den Aufschluss über die grobe Altersgruppe der Passagiere geben.

Nachdem wir die Bedeutung aller Merkmale geklärt haben, wählen wir für die weitere Analyse nur die Merkmale _Passagierklasse_, _Geschlecht_, _Anzahl der Geschwister bzw. Ehepartner an Bord_, _Anzahl der Eltern bzw. Kinder an Bord_, _Passagiertarif_, _die Altersgruppe_ bzw. das Merkmal _who_ aus und speichern ihre Werte im DataFrame _X_ ab. Zur Überprüfung geben wir danach die ersten fünf Zeilen des neuen DataFrames aus.

In [ ]:
# Speichere das zweite (Passagierklasse), das dritte (Geschlecht),
# das fünfte (Anzahl der Geschwister/Ehepartner an Bord), das sechste (Anzahl der Eltern/Kinder an Bord),
# das siebte (Passagiertarif (Britisches Pfund)),
# und das zehnte (Mann/Frau/Kind) Merkmal in die Datenmatrix
X = titanic[['pclass', 'sex', 'sibsp', 'parch', 'fare', 'who']]
print(X.head())

Da wir mit unserem Model die Überlebenschance der Passagiere vorhersagen wollen, ist unser Zielmerkmal der Überlebensstatus. Deswegen speichern wir die Werte des Merkmals _survived_ im DataFrame _y_ ab.

In [ ]:
# Speichere das erste Merkmal (überlebt) als Zielmerkmal
y = titanic[['survived']]
print(y.head())

## One-Hot-Codierung
Da neuronale Netzwerke nur numerische Werte direkt verarbeiten können, müssen kategorische Merkmale in unserem Datensatz vorverarbeitet werden. Wir fangen mit dem Merkmal _who_ an. Eigentlich brauchen wir daraus nur die Information darüber, ob sich bei einem Passagier um ein Kind gehandelt hat, denn die Geschlechtszugehörigkeit der erwachsenen Passagiere im Merkmal _sex_ bereits enthalten ist. Deswegen ersetzen wir alle Einträge im Merkmal _who_, wo früher _child_ stand, mit 1 und alle anderen mit 0. Dabei hilft uns die Funktion _where()_ des _NumPy_-Moduls.

Auf Grund der Änderung der Einträge macht es Sinn, das Merkmal _who_ umzubenennen. Das erreichen wir mit der Funktion _rename()_ des _Pandas_-Moduls. In der Klammer der Funktion spezifizieren wir, dass wir die Spalte bzw. das Merkmal _who_ in _is_child_ umbenennen.

In [ ]:
# Damit unterdrücken wir einige Warnungen
# (Empfehlung von Pandas)
pd.options.mode.copy_on_write = True

# Speichere im Merkmal 'who' nur die Information darüber, ob ein Passagier
# ein Kind war, und benenne es in 'is_child' um
X['who'] = np.where(X['who'] == 'child', 1, 0)
X = X.rename(columns={'who': 'is_child'})
print(X.head())

In unserem Datensatz haben wir noch das Merkmale _sex_, das kategorische Werte enthält. Eine gängige Vorgehensweise kategorische Merkmale in numerische Merkmale umzuwandeln, nennt sich _One-Hot-Codierung_. Bei der One-Hot-Codierung wird für jede Ausprägung eines kategorischen Merkmals ein eigenes Merkmal erstellt, dessen Werte 1 für die Beobachtungen annehmen, für die die Ausprägung des kategorischen Merkmals zutrifft. Für alle anderen Beobachtungen wird 0 im neuen Merkmal gespeichert.

In Python lassen sich die kategorischen Merkmale nach der One-Hot-Methode zum Beispiel mit Hilfe der Funktion _get_dummies()_ des _Pandas_-Moduls umwandeln. In der Funktion müssen wir dem Parameter _columns_ die Liste der Merkmale zuweisen, die umcodiert werden sollen. Außerdem können wir mit Hilfe des Parameters _dtype_ den Datentyp der Einträge der neuen Merkmale festlegen.

In [ ]:
X = pd.get_dummies(X, columns=['sex'], dtype=float)
print(X.head())

Neben dem Merkmal _sex_ müssen wir noch unser Zielmerkmal _survived_ gemäß der One-Hot-Codierung umwandeln, weil das künstliche neuronale Netz für jede Klasse _überlebt_ bzw. _nicht überlebt_ jeweils eine Vorhersage berechnen wird. Dafür nutzen wir wieder die Funktion _get_dummies()_.

In [ ]:
y = pd.get_dummies(y, columns=['survived'], dtype=float)
print(y.head())

## Datennormalisierung
Ein weiterer üblicher Schritt bei der Vorbereitung der Daten ist die Normalisierung. Bei der _Normalisierung_ handelt es sich um eine Methode, bei der die Werte numerischer Merkmale so geändert werden, dass sie die gleiche Größenordnung haben. Meistens werden die Merkmalswerte auf das Intervall zwischen 0 und 1 normalisiert. Wir normalisieren unseren Datensatz mit Hilfe der Funktion <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html"> _minmax_scale()_</a> des _Scikit-learn_-Untermoduls _Preprocessing and Normalization_. Um die originalen Werte des DataFrames nicht zu überschreiben, speichern wir den normalisierten Datensatz im _ndarray_ _X_normalized_ ab.

In [ ]:
# Normalisiere die Merkmalswerte auf den Bereich [0,1]
X_normalized = preprocessing.minmax_scale(X)
print(X_normalized[:1,:])

Wie bei allen Klassifikationsaufgaben muss der Datensatz vor dem Training eines Klassifikationsmodells in eine Trainings- und eine Testmenge zerlegt werden. Dafür benutzen wir die Funktion <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html"> _train_test_split()_</a> des Untermoduls _Model Selection_ von _Scikit-learn_.

In [ ]:
# Teile den Datensatz in die Trainings- und die Testmenge auf
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.15, random_state=61)

## Aufbau eines neuronalen Netzes
In Python können künstliche neuronale Netze mit Hilfe des Moduls <a href="https://keras.io/"> _Keras_</a> schnell und einfach aufgebaut, trainiert und ausgewertet werden. Für die Vorhersage des Überlebens der Passagiere der Titanic implementieren wir ein sequenzielles Feed-Forward-Netz mit sieben Neuronen in der Eingabeschicht, die den sieben Merkmalen unseres Datensatzes entsprechen. Diese verknüpfen wir mit einer vollständig verbundenen Schicht mit 15 Neuronen. Die Ausgabeschicht unseres Netzes enthält zwei Neuronen, die zwei Klassen _überlebt_ und _nicht überlebt_ entsprechen.

Zuerst erzeugen wir ein sequentielles Modell mit dem Befehl _keras.Sequential()_. Wir fügen unserem Modell die _Input-Schicht_ mit der Funktion _add()_ hinzu. Da es sich um die erste Schicht im Netz handelt, müssen wir die Größe der Eingabe dem Parameter _shape_ zuweisen. Nun fügen wir unserem Modell eine verborgene vollständig verbundene bzw. _dense Schicht_ hinzu. Durch die Zuweisung des Parameters _units_ legen wir fest, dass die vollständig verbundene Schicht 15 Neuronen enthalten soll. Außerdem können wir mit Hilfe des Parameters _activation_ die Aktivierungsfunktion festlegen. Hierfür verwenden wir die _ReLU_-Funktion. Schließlich ergänzen wir unser Modell mit der vollständig verbundenen _Ausgabeschicht_ mit zwei Neuronen, eins pro Klasse. Hier verwenden wir die Aktivierungsfunktion _softmax_, deren Vorteil darin besteht, dass ihre Ausgabe sich als Wahrscheinlichkeiten interpretieren lassen, zu bestimmten Klasse zu gehören.

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(X_normalized.shape[1],)))
model.add(keras.layers.Dense(units=15, activation="relu"))
model.add(keras.layers.Dense(2, activation = "softmax"))

## Berechnung der Modellparameter
Nachdem wir das künstliche neuronale Netz implementiert haben, können wir mit der Methode _summary()_ die Zusammenfassung des Modells anzeigen lassen. In der Ausgabe kannst du eine detaillierte Beschreibung einzelner Schichten des Modells inklusive ihrer Ausgaben und der Anzahl der trainierbaren Parameter sehen.

In [ ]:
model.summary()